# Retirement planning with Assistants API (GPTAssistantAgent)

Using AutoGen's `GPTAssistantAgent`, this notebook demonstrates multi-agent retirement planning use case

## Initialize autogen with configuration

Use [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) to load a list of configurations from an environment variable or a json file.

In [ ]:
%pip -qqq install pyautogen

In [ ]:
import autogen
from autogen.agentchat import AssistantAgent
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

## Define retirement planning agent swarm using GroupChat

In [ ]:
# Define user proxy agent
llm_config = {"config_list": config_list, "cache_seed": 45}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A user interested in retirement planning",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="TERMINATE",
)

coder = GPTAssistantAgent(
    name="Coder",
    llm_config={
        "config_list": config_list,
    },
    instructions=AssistantAgent.DEFAULT_SYSTEM_MESSAGE,
)

actuary = GPTAssistantAgent(
    name="Actuary",
    instructions="You are an actuary. You are an expert in risk management and financial planning. You are here to help the user with their retirement planning.",
    llm_config={
        "config_list": config_list,
    },
)

estate_planner = GPTAssistantAgent(
    name="Estate Planner",
    instructions="You are an estate planner. You are an expert in estate planning and wealth management. You are here to help the user with their retirement planning.",
    llm_config={
        "config_list": config_list,
    },
)
# define group chat
groupchat = autogen.GroupChat(agents=[user_proxy, coder, actuary, estate_planner], messages=[], max_round=10)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## Initiate retirement planning
Initiate multi-agent chat with a message to plan for retirement

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="I am 50 years old, expected to retire at 65, and have $500,000 in my retirement account. I want to know how much I need to save each month to retire comfortably.",
)
# type exit to terminate the chat